In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

train_df = pd.read_csv("../input/train.csv")
print("Shape of train data: {0}".format(train_df.shape))
test_df = pd.read_csv("../input/test.csv")
print("Shape of test data: {0}".format(test_df.shape))

diagnosis_df = pd.DataFrame({
    'diagnosis': [0, 1, 2, 3, 4],
    'diagnosis_label': ['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']
})

train_df = train_df.merge(diagnosis_df, how="left", on="diagnosis")

train_image_files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("../input/train_images")) for f in fn]
train_images_df = pd.DataFrame({
    'files': train_image_files,
    'id_code': [file.split('/')[3].split('.')[0] for file in train_image_files],
})
train_df = train_df.merge(train_images_df, how="left", on="id_code")
del train_images_df
print("Shape of train data: {0}".format(train_df.shape))

test_image_files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("../input/test_images")) for f in fn]
test_images_df = pd.DataFrame({
    'files': test_image_files,
    'id_code': [file.split('/')[3].split('.')[0] for file in test_image_files],
})


test_df = test_df.merge(test_images_df, how="left", on="id_code")
del test_images_df
print("Shape of test data: {0}".format(test_df.shape))

# Any results you write to the current directory are saved as output.


['train.zip', 'test_images', 'sample_submission.csv.zip', 'description.md', 'test_images.zip', 'train_images', 'train_images.zip', 'test.zip', 'sample_submission.csv', 'test.csv.zip', 'train.csv', 'train.csv.zip', 'test.csv', 'aptos2019-blindness-detection']
Shape of train data: (3295, 2)
Shape of test data: (367, 1)
Shape of train data: (3295, 4)
Shape of test data: (367, 2)


In [3]:
train_df.head()


,id_code,diagnosis,diagnosis_label,files
0,2a2274bcb00a,0,No DR,../input/train_images/2a2274bcb00a.png
1,eda29a9d78f3,0,No DR,../input/train_images/eda29a9d78f3.png
2,789c60cba801,0,No DR,../input/train_images/789c60cba801.png
3,a07efb1ecfc0,0,No DR,../input/train_images/a07efb1ecfc0.png
4,44855f666225,2,Moderate,../input/train_images/44855f666225.png


In [4]:
test_df.head()


,id_code,files
0,b460ca9fa26f,../input/test_images/b460ca9fa26f.png
1,6cee2e148520,../input/test_images/6cee2e148520.png
2,ca6842bfcbc9,../input/test_images/ca6842bfcbc9.png
3,6cbc3dad809c,../input/test_images/6cbc3dad809c.png
4,a9bc2f892cb3,../input/test_images/a9bc2f892cb3.png


In [5]:
IMG_SIZE = 150
N_CLASSES = train_df.diagnosis.nunique()
CLASSES = list(map(str, range(N_CLASSES)))
BATCH_SIZE = 32
EPOCH_STEPS = 10
EPOCHS = 1


In [6]:
import tensorflow as tf
print(tf.__version__)

from keras.preprocessing.image import ImageDataGenerator

train_df["diagnosis"] = train_df["diagnosis"].astype(str)

train_data_gen = ImageDataGenerator(rescale=1.0/255, validation_split=0.3)
train_data = train_data_gen.flow_from_dataframe(
    dataframe=train_df, 
    x_col="files",
    y_col="diagnosis",
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=CLASSES,
    class_mode="sparse",
    target_size=(IMG_SIZE, IMG_SIZE),
    subset="training"
)

validation_data = train_data_gen.flow_from_dataframe(
    dataframe=train_df, 
    x_col="files",
    y_col="diagnosis",
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=CLASSES,
    class_mode="sparse",
    target_size=(IMG_SIZE, IMG_SIZE),
    subset="validation"
)

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_dataframe(
    dataframe=test_df,
    x_col="files",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size = 1,
    shuffle=False,
    class_mode=None
)


2025-08-18 20:52:31.332527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755550351.345921       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755550351.350132       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 20:52:31.368283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.11/dist-packages/keras/api/preprocessing/image/__init__.py)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(N_CLASSES, activation='softmax')
])
opt = tf.keras.optimizers.Adam(lr=0.001, epsilon=1e-6)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-18 20:52:37.189259: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: Argument(s) not recognized: {'lr': 0.001}

In [8]:
model.fit(
    train_data,
    steps_per_epochs=EPOCH_STEPS,
    epochs=EPOCHS,
    validation_data=validation_data,
    validation_steps=10
)


NameError: name 'train_data' is not defined

In [9]:
filenames = test_data.filenames
classifications = model.predict_generator(test_data, steps=len(filenames))


NameError: name 'test_data' is not defined

In [10]:
results = pd.DataFrame({
    "id_code": filenames,
    "diagnosis": np.argmax(classifications, axis=1)
})
results["id_code"] = results["id_code"].map(lambda x: str(x)[:-4].split("/")[3])
results.head()


NameError: name 'filenames' is not defined

In [11]:
results.to_csv("submission.csv", index=False)


NameError: name 'results' is not defined